In [15]:
from pathlib import Path
import numpy as np
import datetime
import netCDF4
import xarray as xr
import matplotlib.pyplot as plt
import pyproj
from pyproj import Transformer
from pyproj import CRS
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import hvplot.xarray

# import cartopy.feature as cfe
# import cartopy.crs as ccrs
# import pandas as pd

# Functions

In [9]:
def read_file_paths(filename):
    this_dir=Path.cwd()
    data_dir=this_dir.parent/Path("Data")
    files = data_dir.rglob(filename)
    for i in files:
        data = i
    return data

        
def find_racmo_for_insitu_data(dataset,v,in_situ_site):
    data_v= dataset[v]
    time_series = data_v.sel(rlon=in_situ_site[0], rlat=in_situ_site[1], method='nearest')
    return time_series      

def find_anomaly(dataset):
    return dataset - dataset.mean(dim='time')

# Constants

In [10]:
data_crs=CRS('-m 57.295779506 +proj=ob_tran +o_proj=latlon +o_lat_p=-180.0 +lon_0=30.0 +o_lon_p=0')
from_lat_lon_crs = CRS.from_proj4("+proj=latlon")
transproj = Transformer.from_crs(from_lat_lon_crs, data_crs)

#SPRI AWS STATION COORDINATE
AWSlon = -67.69971
AWSlat = -71.14409

#BAS FOSSIL BLUFF AWS STATION COORDINATE
BAS_FB_lon = -68
BAS_FB_lat = -71.329

##SPRI AWS STATION ROTATED
aws_rotated=transproj.transform(AWSlon,AWSlat)
print('AWS rotated = ',aws_rotated)


##BAS FOSSIL BLUFF AWS STATION ROTATED
bas_fb_rotated=transproj.transform(BAS_FB_lon,BAS_FB_lat)
print('BAS Fossil Bluff rotated = ',bas_fb_rotated)

AWS rotated =  (-18.69776623671606, -2.481756537563884)
BAS Fossil Bluff rotated =  (-18.501719991296497, -2.5536000532157934)


# Read all data

In [11]:
snowmelt_data=xr.open_dataset(read_file_paths('snowmelt.nc')) 
precip_data=xr.open_dataset(read_file_paths('precipitation.nc')) 
rh_data=xr.open_dataset(read_file_paths('relative_humidity.nc')) 
temp_data=xr.open_dataset(read_file_paths('temp.nc')) 

# Create RACMO's outputs for insitu sites

In [12]:
aws_melt=find_racmo_for_insitu_data(snowmelt_data,'snowmelt',aws_rotated)
fb_melt=find_racmo_for_insitu_data(snowmelt_data,'snowmelt',bas_fb_rotated)

aws_temp=find_racmo_for_insitu_data(temp_data,'t2m',aws_rotated)
fb_temp=find_racmo_for_insitu_data(temp_data,'t2m',bas_fb_rotated)

aws_rh=find_racmo_for_insitu_data(rh_data,'rh2m',aws_rotated)
fb_rh=find_racmo_for_insitu_data(rh_data,'rh2m',bas_fb_rotated)

aws_prec=find_racmo_for_insitu_data(precip_data,'precip',aws_rotated)
fb_prec=find_racmo_for_insitu_data(precip_data,'precip',bas_fb_rotated)

C:\Users\Pearl\anaconda3\envs\dissenv\lib\site-packages\xarray\core\indexes.py:234: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  indexer = self.index.get_loc(
C:\Users\Pearl\anaconda3\envs\dissenv\lib\site-packages\xarray\core\indexes.py:234: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  indexer = self.index.get_loc(
C:\Users\Pearl\anaconda3\envs\dissenv\lib\site-packages\xarray\core\indexes.py:234: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  indexer = self.index.get_loc(
C:\Users\Pearl\anaconda3\envs\dissenv\lib\site-packages\xarray\core\indexes.py:234: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version.

# Create Anomaly Datasets

In [27]:
melt_anomaly=find_anomaly(snowmelt_data)
temp_anomaly=find_anomaly(temp_data)
rh_anomaly=find_anomaly(rh_data)
prec_anomaly=find_anomaly(precip_data)

# Try Time Slider

In [48]:
temp_data.t2m.hvplot()

:NdOverlay   [Element]
   :Histogram   [t2m]   (t2m_count)

In [60]:
x=temp_data.t2m.squeeze()

In [61]:
x.hvplot(groupby="time",
                 clim=(240, 290),
                 widget_type="scrubber",
                 widget_location="bottom")

Column
    [0] HoloViews(DynamicMap, widget_location='bottom', widget_type='scrubber')
    [1] Row
        [0] HSpacer()
        [1] WidgetBox
            [0] Player(end=7767, width=550)
        [2] HSpacer()

In [ ]:
temp_anomaly.t2m.hvplot()

In [63]:
x=temp_anomaly.t2m.squeeze()
x.hvplot(groupby="time",
                 clim=(-23, 11),
                 widget_type="scrubber",
                 widget_location="bottom")

Column
    [0] HoloViews(DynamicMap, widget_location='bottom', widget_type='scrubber')
    [1] Row
        [0] HSpacer()
        [1] WidgetBox
            [0] Player(end=7767, width=550)
        [2] HSpacer()